# 제32회 ADP 실기 대비 - 핵심만 요약한 통계와 머신러닝 파이썬 코드북

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 설정

# Colab 한글 폰트 설정
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# plt.rcParams['font.family'] = 'NanumBarunGothic'

# 3장. 표본추출, 데이터 분할, 교차검증

## 3-1. 표본추출
- 표본추출 방법으로는 단순랜덤추출법, 계통추출법, 집략추출법, 층화추출법이 있다.

### 단순랜덤추출법 (Simple random sampling)
- 각 샘플에 번호를 부여하여 임의의 n개를 추출하는 방법으로 각 샘플은 선택될 확률이 동일하다.
- 추출한 요소(Element)를 다시 집어 넣어 추출하면 복원 추출, 다시 집어넣지 않고 추출하면 비복원 추출이다.
- 단순랜덤추출을 하기 위해 Pandas, random, Numpy를 사용할 수 있다.
- 다음에서 scikit-learn의 내장 데이터 'iris'에 대해 여러가지 방법으로 단순랜덤추출을 해보고자 한다.

In [2]:
# scikit-learn의 내장 데이터 iris 불러오기
from sklearn.datasets import load_iris
data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=[col.replace(" (cm)", "") for col in iris_cols])
iris.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [3]:
# 복원하지 않고 3개를 랜덤추출
iris.sample(n=3, replace=False)

,sepal length,sepal width,petal length,petal width,target
73,6.1,2.8,4.7,1.2,1.0
99,5.7,2.8,4.1,1.3,1.0
103,6.3,2.9,5.6,1.8,2.0


In [4]:
# frac 파라미터로 전체 데이터의 3%를 랜덤추출
iris.sample(frac=0.03)

,sepal length,sepal width,petal length,petal width,target
13,4.3,3.0,1.1,0.1,0.0
112,6.8,3.0,5.5,2.1,2.0
142,5.8,2.7,5.1,1.9,2.0
101,5.8,2.7,5.1,1.9,2.0


In [5]:
# 3개의 열(axis=1)을 랜덤 추출
iris.sample(3, axis=1).head(3)

,target,petal length,petal width
0,0.0,1.4,0.2
1,0.0,1.4,0.2
2,0.0,1.3,0.2


In [6]:
# 리스트에서 단순랜덤추출하기
import random
data_list = [1, 2, 3, 4, 5, 'a', 'b', 'c']

print(f"random.sample: {random.sample(data_list, 4)}")
print(f"np.random.choice: {np.random.choice(data_list, 4, replace=True)}")

random.sample: [2, 'a', 5, 4]
np.random.choice: ['2' '4' 'c' '4']


In [7]:
# [참고] 난수 생성하기
print(f"0~10 사이의 정수 중 3개의 난수 생성: {np.random.randint(0, 10, 3)}")
print(f"0~1 사이의 실수를 2*2 배열로 생성: {np.random.rand(2, 2)}")

0~10 사이의 정수 중 3개의 난수 생성: [4 4 7]
0~1 사이의 실수를 2*2 배열로 생성: [[0.4427665  0.82667265]
 [0.66012251 0.90474853]]


### 계통추출법 (Systematic sampling)
- 먼저, 번호를 부여한 샘플을 나열한다.
- 총 N(30)개의 모집단에서 n(5)개의 샘플을 추출하기 위해서 N/n으로 구간을 나눈다.
- 이 경우, 각 구간에 들어있는 샘플의 수는 K(6)가 된다.
- K(6)개의 샘플이 들어있는 첫 구간에서 임의로 샘플을 하나 선택하고, K(6)개씩 띄어서 각 구간에서 하나씩 샘플을 추출한다.

In [8]:
# 계통추출법으로 표본추출
data, n = iris, 8 # 모집단 데이터프레임, 추출할 샘플 수
N = len(data) # 모집단 데이터 크기
K = N // n # 구간 내 샘플 수

index = data[:K].sample(1).index # 첫 구간에서 임의로 선택한 샘플 인덱스

# 첫 샘플로부터 K개씩 띄어서 각 구간에서 하나씩 샘플을 추출
sys_df = pd.DataFrame()

while len(sys_df) < n:
    sys_df = sys_df.append(data.loc[index, :])
    index += K
    
print(f"N: {N}")
print(f"n: {n}")
print(f"K: {K}")
sys_df

N: 150
n: 8
K: 18


,sepal length,sepal width,petal length,petal width,target
9,4.9,3.1,1.5,0.1,0.0
27,5.2,3.5,1.5,0.2,0.0
45,4.8,3.0,1.4,0.3,0.0
63,6.1,2.9,4.7,1.4,1.0
81,5.5,2.4,3.7,1.0,1.0
99,5.7,2.8,4.1,1.3,1.0
117,7.7,3.8,6.7,2.2,2.0
135,7.7,3.0,6.1,2.3,2.0


### 집략추출법 (Cluster random sampling)
- 군집별로 랜덤추출법을 수행하여 표본을 얻는 방법이다.
- 지역표본추출과 다단계표본추출이 이에 해당한다.
- 이 경우, 군집 내 요소들은 상이하지만, 군집과 군집은 비교적 유사한 특성을 띈다.
- 아래 층화추출법에서 층을 집략으로 대치하면 된다.

### 층화추출법 (Stratified random sampling)
- 계층별로 랜덤추출법을 수행하여 표본을 얻는 방법이다.
- 비례층화추출법과 불비례층화추출법이 이에 해당한다.
- 각 층(Stratum) 내 요소들은 유사하지만, 층과 층의 요소들은 상이하다.

In [9]:
# target을 층 혹은 집략이라고 가정
# 원본 데이터의 분포 확인

iris['target'].value_counts()

2.0    50
1.0    50
0.0    50
Name: target, dtype: int64

In [10]:
# 데이터, 층/집략 정보를 가진 컬럼명, 추출표본 개수
data, stratum, sampling_no = iris, 'target', 9

# 비례층화추출법: 원본 데이터의 비율대로 추출
levels = data[stratum].unique()
total = data[stratum].value_counts().sum()
prop_val = data[stratum].value_counts() / total
no = prop_val * sampling_no

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = pd.concat([result, temp_df])
    
result

,sepal length,sepal width,petal length,petal width,target
12,4.8,3.0,1.4,0.1,0.0
18,5.7,3.8,1.7,0.3,0.0
2,4.7,3.2,1.3,0.2,0.0
60,5.0,2.0,3.5,1.0,1.0
99,5.7,2.8,4.1,1.3,1.0
51,6.4,3.2,4.5,1.5,1.0
101,5.8,2.7,5.1,1.9,2.0
103,6.3,2.9,5.6,1.8,2.0
125,7.2,3.2,6.0,1.8,2.0


In [11]:
# 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
# 데이터, 총/집략 정보를 가진 컬럼명, 추출표본 개수, 각 층/집략의 비율
data, stratum, sampling_no, proportion = iris, 'target', 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
total = sum(prop_val)

no = prop_val * sampling_no

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = pd.concat([result, temp_df])
    
result

,sepal length,sepal width,petal length,petal width,target
47,4.6,3.2,1.4,0.2,0.0
30,4.8,3.1,1.6,0.2,0.0
86,6.7,3.1,4.7,1.5,1.0
99,5.7,2.8,4.1,1.3,1.0
82,5.8,2.7,3.9,1.2,1.0
76,6.8,2.8,4.8,1.4,1.0
75,6.6,3.0,4.4,1.4,1.0
140,6.7,3.1,5.6,2.4,2.0
143,6.8,3.2,5.9,2.3,2.0
118,7.7,2.6,6.9,2.3,2.0


## 3-2. 데이터 분할
- 데이터를 학습용과 시험용으로 분할하여 학습하고 검증하는 이유는, 과적합을 피함으로써 일반화된 성능을 확보하기 위함이다.
- 데이터 분할 방법에는 일반적인 데이터 분할 및 홀드아웃방법이 있고, Shuffle split, K-fold, 층화 K-fold, Group K-fold, Stratified Group K-fold 등이 있다.

### 일반적 데이터 분할 및 홀드아웃 방법
- 일반적으로는 학습용(Train set) 70%, 검증용(Test set) 30%의 비율에 따라 랜덤으로 데이터를 분할하여 모델 학습에 사용한다.
- 여기서 홀드아웃(Hold-out) 방법이란, 학습용과 검증용의 비율을 50:50으로 하는 것을 말한다.
- scikit-learn의 train_test_split의 파라미터 test_size에서 test set의 비율을 지정함으로써 간단하게 데이터를 분할할 수 있다.
- random_state에 상수를 입력하지 않으면, 코드를 실행할 때마다 다른 구성으로 데이터가 분할되며, 상수를 입력하면 매번 동일한 구성으로 데이터가 분할된다.

In [12]:
from sklearn.model_selection import train_test_split

X = iris.drop('target', axis=1)
y = iris.filter(['target'])

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(f"X_train(y_train): {len(X_train), len(y_train)}, X_test(y_test): {len(X_test), len(y_test)}")
print(f"X_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")

X_train(y_train): (105, 105), X_test(y_test): (45, 45)
X_train의 비율: 0.7, X_test의 비율: 0.3


In [13]:
# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print(f"X_train(y_train): {len(X_train), len(y_train)}, X_test(y_test): {len(X_test), len(y_test)}")
print(f"X_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")

X_train(y_train): (75, 75), X_test(y_test): (75, 75)
X_train의 비율: 0.5, X_test의 비율: 0.5


### Shuffle split
- 무작위 순열 교차 검증(Random permutation cross-validate)에 사용한다.
- 데이터 크기가 작은 경우, 분할 샘플들이 유사할 수도 있다.
- scikit-learn의 ShuffleSplit을 사용해서 간단하게 무작위 순열 교차 검증을 진행할 수 있다.
- 파라미터 train_size와 test_size에 전체 데이터 대비 학습용 데이터와 검증용 데이터의 비율을 입력함으로써 각 데이터의 크기를 설정하고, n_splits로 샘플 분할 횟수를 지정한다.

In [14]:
from sklearn.model_selection import ShuffleSplit
from collections import Counter

ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=4)
for i, (train_index, test_index) in enumerate(ss.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print(f"\tX_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")
    print(f"\ty_train의 타겟 구성: {Counter(y_train['target'])}")
    print(f"\ty_test의 타겟 구성: {Counter(y_test['target'])}")

Sample 0 ==> train_index: [ 16 136  15], test_index: [26 55  0]
	X_train의 비율: 0.5, X_test의 비율: 0.5
	y_train의 타겟 구성: Counter({2.0: 29, 1.0: 24, 0.0: 22})
	y_test의 타겟 구성: Counter({0.0: 28, 1.0: 26, 2.0: 21})
Sample 1 ==> train_index: [88 80 54], test_index: [ 43 110   9]
	X_train의 비율: 0.5, X_test의 비율: 0.5
	y_train의 타겟 구성: Counter({1.0: 28, 2.0: 27, 0.0: 20})
	y_test의 타겟 구성: Counter({0.0: 30, 2.0: 23, 1.0: 22})
Sample 2 ==> train_index: [95  8 92], test_index: [ 30 103  76]
	X_train의 비율: 0.5, X_test의 비율: 0.5
	y_train의 타겟 구성: Counter({0.0: 28, 2.0: 26, 1.0: 21})
	y_test의 타겟 구성: Counter({1.0: 29, 2.0: 24, 0.0: 22})
Sample 3 ==> train_index: [67 53  8], test_index: [127 138 132]
	X_train의 비율: 0.5, X_test의 비율: 0.5
	y_train의 타겟 구성: Counter({1.0: 27, 0.0: 25, 2.0: 23})
	y_test의 타겟 구성: Counter({2.0: 27, 0.0: 25, 1.0: 23})


### K-fold 분할
- 데이터를 K개의 집단으로 분할한 뒤 (K-1)개의 집단의 데이터를 학습용 데이터로 삼고, 나머지 1개 집단의 데이터를 검증용 데이터로 삼는 방법이다.
- 이 과정을 K번 반복하여 모든 데이터가 학습과 검증에 사용될 수 있도록 하고, 이 과정에서 얻은 여러 평균제곱오차(MSE)들의 평균을 해당 모델의 MSE 값으로 사용한다.
- 학습용 데이터와 검증용 데이터의 크기는 fold를 몇 개로 정하는지에 따라 정해진다.
- 5-fold이면, 학습용은 전체 데이터의 4/5, 검증용 데이터는 전체 데이터의 1/5이 되고, 10-fold이면, 학습용은 전체 데이터의 9/10, 검증용 데이터는 전체 데이터의 1/10이 된다.

In [15]:
from sklearn.model_selection import KFold
from collections import Counter

# n_splits=fold의 개수, shuffle=데이터 분할 전 shuffle 여부
kf = KFold(n_splits=4, shuffle=False)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print(f"\tX_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")
    print(f"\ty_train의 타겟 구성: {Counter(y_train['target'])}")
    print(f"\ty_test의 타겟 구성: {Counter(y_test['target'])}")

Sample 0 ==> train_index: [38 39 40], test_index: [0 1 2]
	X_train의 비율: 0.7466666666666667, X_test의 비율: 0.25333333333333335
	y_train의 타겟 구성: Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟 구성: Counter({0.0: 38})
Sample 1 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.7466666666666667, X_test의 비율: 0.25333333333333335
	y_train의 타겟 구성: Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 12})
Sample 2 ==> train_index: [0 1 2], test_index: [76 77 78]
	X_train의 비율: 0.7533333333333333, X_test의 비율: 0.24666666666666667
	y_train의 타겟 구성: Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟 구성: Counter({1.0: 24, 2.0: 13})
Sample 3 ==> train_index: [0 1 2], test_index: [113 114 115]
	X_train의 비율: 0.7533333333333333, X_test의 비율: 0.24666666666666667
	y_train의 타겟 구성: Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟 구성: Counter({2.0: 37})


### Stratified K-fold 분할
- K-fold 방법으로 분류용 데이터를 분할하는 과정에서, 어떤 분할에서는 학습용 데이터나 검증용 데이터에 타겟 변수의 일부 클래스가 포함되지 않거나 클래스 불균형인 채로 분할되는 경우가 생길 수 있다.
- 이를 방지하기 위해 데이터를 K개의 집단으로 나눌 때, 타겟 변수의 클래스들이 각 fold 별로 일정한 비율로 배치되도록 하는 것이 Stratified K-fold 분할 방법이다.
- 앞서 살펴 본 K-fold 예제 속 샘플들의 타겟 구성은 불균형인 반면, 다음의 Stratified K-fold 샘플들의 타겟 구성은 비교적 균형 잡힌 것을 볼 수 있다.

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4)

# 분할 시 y를 고려해야 하기 때문에 split에 y를 입력
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print(f"\tX_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")
    print(f"\ty_train의 타겟 구성: {Counter(y_train['target'])}")
    print(f"\ty_test의 타겟 구성: {Counter(y_test['target'])}")

Sample 0 ==> train_index: [13 14 15], test_index: [0 1 2]
	X_train의 비율: 0.7466666666666667, X_test의 비율: 0.25333333333333335
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 1 ==> train_index: [0 1 2], test_index: [13 14 15]
	X_train의 비율: 0.7466666666666667, X_test의 비율: 0.25333333333333335
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12})
Sample 2 ==> train_index: [0 1 2], test_index: [26 27 28]
	X_train의 비율: 0.7533333333333333, X_test의 비율: 0.24666666666666667
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})
Sample 3 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.7533333333333333, X_test의 비율: 0.24666666666666667
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12})


### Group K-fold 분할
- 범주형 변수인 group의 수준별 데이터들을 각 분할마다 검증용 데이터로 사용하도록 K-fold를 진행하는 방법이다.
- 이 때문에 group의 수준의 개수는 fold의 개수와 같거나 fold 개수보다 커야 한다.
- 다음 예제에서 4개의 수준을 가지는 group 변수를 임의로 생성하여 Group K-fold 분할을 진행하였다.
- 각 검증용 데이터는 1종류의 그룹으로 구성됨을 알 수 있다.

In [17]:
# group 변수가 있는 데이터를 생성
## group의 수준은 g0, g1, g2, g3의 4종류가 있다.
iris2 = iris.copy()
iris2['group'] = iris2['target'].apply(lambda x: f"g{int(np.random.randint(0, 4, 1))}")
print(Counter(iris2['group']))
iris2.head(3)

Counter({'g3': 44, 'g1': 37, 'g2': 36, 'g0': 33})


,sepal length,sepal width,petal length,petal width,target,group
0,5.1,3.5,1.4,0.2,0.0,g1
1,4.9,3.0,1.4,0.2,0.0,g3
2,4.7,3.2,1.3,0.2,0.0,g3


In [18]:
from sklearn.model_selection import GroupKFold
X = iris2.drop(['target', 'group'], axis=1)
y = iris2.filter(['target'])

group = iris2.filter(['group'])
gkf = GroupKFold(n_splits=4)

# 분할 시 group을 고려해야 하기 때문에 split에 group를 입력
for i, (train_index, test_index) in enumerate(gkf.split(X, y, group)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print(f"\tX_train의 비율: {len(X_train) / len(X)}, X_test의 비율: {len(X_test) / len(X)}")
    print(f"\ty_train의 타겟 구성: {Counter(y_train['target'])}")
    print(f"\ty_test의 타겟 구성: {Counter(y_test['target'])}")
    print(f"\ty_train의 그룹 구성: {Counter(group.iloc[train_index]['group'])}")
    print(f"\ty_test의 그룹 구성: {Counter(group.iloc[test_index]['group'])}")

Sample 0 ==> train_index: [0 3 4], test_index: [1 2 5]
	X_train의 비율: 0.7066666666666667, X_test의 비율: 0.29333333333333333
	y_train의 타겟 구성: Counter({0.0: 38, 1.0: 35, 2.0: 33})
	y_test의 타겟 구성: Counter({2.0: 17, 1.0: 15, 0.0: 12})
	y_train의 그룹 구성: Counter({'g1': 37, 'g2': 36, 'g0': 33})
	y_test의 그룹 구성: Counter({'g3': 44})
Sample 1 ==> train_index: [1 2 4], test_index: [0 3 6]
	X_train의 비율: 0.7533333333333333, X_test의 비율: 0.24666666666666667
	y_train의 타겟 구성: Counter({1.0: 40, 0.0: 37, 2.0: 36})
	y_test의 타겟 구성: Counter({2.0: 14, 0.0: 13, 1.0: 10})
	y_train의 그룹 구성: Counter({'g3': 44, 'g2': 36, 'g0': 33})
	y_test의 그룹 구성: Counter({'g1': 37})
Sample 2 ==> train_index: [0 1 2], test_index: [ 4 11 16]
	X_train의 비율: 0.76, X_test의 비율: 0.24
	y_train의 타겟 구성: Counter({0.0: 40, 2.0: 40, 1.0: 34})
	y_test의 타겟 구성: Counter({1.0: 16, 0.0: 10, 2.0: 10})
	y_train의 그룹 구성: Counter({'g3': 44, 'g1': 37, 'g0': 33})
	y_test의 그룹 구성: Counter({'g2': 36})
Sample 3 ==> train_index: [0 1 2], test_index: [7 8 9]
	X_train

## 3-3. 교차 검증
- 특정한 학습용 데이터와 검증용 데이터에만 최적화되도록 모델을 만들어가면 해당 데이터 세트에만 잘 동작하는 모델이 만들어질 수 있다.
- 따라서 학습용 데이터와 검증용 데이터를 앞서 언급한 분할 방법들로 다양하게 분할하고, 여러 파라미터 조건 하에 학습하여 교차 검증함으로써 최적화된 일반화 모델을 완성할 수 있다.

### 분할 샘플들로 교차 검증
- 데이터 구성이 다른 샘플들을 통해 교차 검증(Cross validation)을 할 수 있다.
- scikit-learn의 cross_validate을 사용하면 각 케이스의 적합 소요 시간, 검증 소요 시간, 테스트셋 스코어, 트레인셋 스코어가 반환된다.

In [19]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression

X = iris.drop('target', axis=1)
y = iris['target']

LOGREG = LogisticRegression(max_iter=300, C=0.1) # 학습 모델 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의

result = cross_validate(LOGREG, X, y, cv=SKF, return_train_score=True) # 교차 검증
pd.DataFrame(result)

,fit_time,score_time,test_score,train_score
0,0.009995,0.001000,0.894737,0.955357
1,0.003809,0.000000,0.947368,0.964286
2,0.016345,0.000000,0.945946,0.955752
3,0.000000,0.013809,1.000000,0.938053


### 파라미터 후보들로 교차 검증
- 분할된 샘플들뿐만 아니라 파라미터 후보들을 포함하여 교차 검증을 진행할 수도 있다.
- 파라미터 별 평균 적합시간, 적합 시간 편차, 분할별 테스트 스코어, 평균 테스트 스코어, 테스트 스코어 편차, 테스트 스코어 랭킹 등을 반환한다.
- 최적의 매개변수 조합은 best_params_를 통해 확인할 수 있다.

In [20]:
from sklearn.model_selection import GridSearchCV

LOGREG = LogisticRegression(max_iter=300) # 학습 모델 정의
param_grid = {'C' : [0.01, 0.1, 1], 'solver' : ['lbfgs', 'liblinear']} # 파라미터 후보 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의

grid = GridSearchCV(LOGREG, param_grid, cv=SKF) # 굧 ㅏ검증
grid.fit(X, y)

print(f"최상의 교차 검증 정수: {grid.best_score_}")
print(f"최적의 매개 변수: {grid.best_params_}")
pd.DataFrame(grid.cv_results_)

최상의 교차 검증 정수: 0.9733285917496444
최적의 매개 변수: {'C': 1, 'solver': 'lbfgs'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007497,0.005245,0.000501,0.000501,0.01,lbfgs,"{'C': 0.01, 'solver': 'lbfgs'}",0.815789,0.894737,0.783784,0.945946,0.860064,0.063947,4
1,0.001001,0.000005,0.000501,0.000501,0.01,liblinear,"{'C': 0.01, 'solver': 'liblinear'}",0.684211,0.684211,0.648649,0.648649,0.666430,0.017781,6
2,0.007245,0.000841,0.000749,0.000432,0.1,lbfgs,"{'C': 0.1, 'solver': 'lbfgs'}",0.894737,0.947368,0.945946,1.000000,0.947013,0.037221,3
3,0.000806,0.000477,0.000566,0.000395,0.1,liblinear,"{'C': 0.1, 'solver': 'liblinear'}",0.815789,0.842105,0.837838,0.783784,0.819879,0.023109,5
4,0.007544,0.007077,0.004270,0.006255,1,lbfgs,"{'C': 1, 'solver': 'lbfgs'}",0.973684,0.973684,0.945946,1.000000,0.973329,0.019114,1
5,0.000000,0.000000,0.003778,0.006544,1,liblinear,"{'C': 1, 'solver': 'liblinear'}",1.000000,0.947368,0.864865,1.000000,0.953058,0.055266,2


## 연습문제

#### 1.
- 다음은 포르투갈 은행의 정기예금 프로모션 전화 데이터이다.
- 데이터는 고객의 특징을 나타내는 입력 변수들과 고객이 정기예금에 가입했는지 여부를 나타내는 출력 변수로 구성되어 있다.
- 데이터 컬럼 정의서는 아래와 같다.
- 다음 데이터를 랜덤 포레스트 알고리즘의 파라미터 max_depth의 후보 4개를 토대로 교차분석을 진행하고, 그 결과를 표로 나타내시오.

|변수명|설명|
|-|-|
|age|나이|
|job|직업의 형태|
|martial|결혼 상태|
|education|학력|
|default|신용 불이행 여부|
|balance|은행 잔고|
|housing|부동산 대출 여부|
|loan|개인 대출 여부|
|contact|연락 수단|
|month|마지막으로 연락한 달|
|y|고객이 정기예금에 가입했는지 여부|

In [21]:
df = pd.read_csv("https://raw.githubusercontent.com/algoboni/pythoncodebook1-1/main/practice1_bank.csv")
df.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,month,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,oct,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,may,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,apr,no


In [22]:
# 범주형 변수들을 레이블 인코딩을 통해 전처리 한다.
df2 = df.copy()

for col in [i for i in df.columns if df[i].dtypes==object]:
    DICT = dict(zip(df[col].unique(), [i for i in range(df[col].nunique())]))
    df2[col] = df2[col].map(DICT)
    
# 마지막 변수의 DICT 확인
print(df[col].unique())
print([i for i in range(df[col].nunique())])
print(DICT)

['no' 'yes']
[0, 1]
{'no': 0, 'yes': 1}


In [23]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier() # 학습 모델 정의
param_grid = {'max_depth' : [5, 6, 7, 8]}
SKF = StratifiedKFold(n_splits=6) # 데이터 분할 방법 정의
grid = GridSearchCV(rf, param_grid, cv=SKF) # 교차 검증

X = df2.drop('y', axis=1)
y = df2['y']
grid.fit(X, y)

print(f"최상의 교차 검증 점수: {round(grid.best_score_, 2)}")
print(f"최적의 매개 변수: {grid.best_params_}")

pd.DataFrame(grid.cv_results_)

최상의 교차 검증 점수: 0.88
최적의 매개 변수: {'max_depth': 7}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.158836,0.012711,0.008986,0.004832,5,{'max_depth': 5},0.884615,0.884615,0.883289,0.885790,0.884462,0.884462,0.884539,0.000725,2
1,0.170323,0.012539,0.010328,0.006826,6,{'max_depth': 6},0.887268,0.881963,0.883289,0.885790,0.885790,0.881806,0.884318,0.002080,3
2,0.174571,0.020913,0.007859,0.004607,7,{'max_depth': 7},0.887268,0.881963,0.883289,0.885790,0.887118,0.883134,0.884760,0.002063,1
3,0.173310,0.005504,0.011759,0.006194,8,{'max_depth': 8},0.888594,0.884615,0.879310,0.883134,0.887118,0.881806,0.884096,0.003128,4
